In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
warnings.filterwarnings('ignore')

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v10'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# GPU Check
import torch
GPU_AVAILABLE = torch.cuda.is_available()

print("="*70)
print("🚀 FOREX SIGNAL GENERATOR V10")
print("   Target: Beat V8 (75%+ = 67.6%, 85%+ = 93.8%)")
print("="*70)
print(f"✓ GPU Available: {GPU_AVAILABLE}")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V10
   Target: Beat V8 (75%+ = 67.6%, 85%+ = 93.8%)
✓ GPU Available: True
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\ml_models_train\models\signal_generator_v10


## 1. Data Loading

In [2]:
# Load Data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

for df in [train_df, test_df]:
    if 'timestamp' in df.columns:
        df.rename(columns={'timestamp': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")
print(f"Train period: {train_df['time'].min()} to {train_df['time'].max()}")
print(f"Test period: {test_df['time'].min()} to {test_df['time'].max()}")

Train: 1,859,492 rows
Test: 296,778 rows
Train period: 2019-12-31 16:00:00+00:00 to 2024-12-30 16:00:00+00:00
Test period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. V10 Feature Engineering (V8 base + Enhanced)

In [3]:
def add_features_v10(df):
    """
    V10 Features: V8-ийн амжилттай features + Шинэ pattern features
    """
    df = df.copy()
    
    # ==================== V8 CORE FEATURES ====================
    # Time Features
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    
    # Moving Averages
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    
    # ADX
    df['tr0'] = abs(df['high'] - df['low'])
    df['tr1'] = abs(df['high'] - df['close'].shift())
    df['tr2'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    
    period = 14
    df['atr'] = df['tr'].rolling(period).mean()
    df['plus_di'] = 100 * (df['plus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['minus_di'] = 100 * (df['minus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'] + 1e-10)
    df['adx'] = df['dx'].rolling(period).mean()
    
    # CCI
    tp = (df['high'] + df['low'] + df['close']) / 3
    sma_tp = tp.rolling(20).mean()
    mad_tp = tp.rolling(20).apply(lambda x: np.abs(x - x.mean()).mean())
    df['cci'] = (tp - sma_tp) / (0.015 * mad_tp + 1e-10)
    
    # Williams %R
    hh = df['high'].rolling(14).max()
    ll = df['low'].rolling(14).min()
    df['williams_r'] = -100 * (hh - df['close']) / (hh - ll + 1e-10)
    
    # Volatility
    df['returns'] = df['close'].pct_change()
    df['volatility'] = df['returns'].rolling(20).std() * 100
    
    # V8 Composite Features
    df['rsi_x_adx'] = df['rsi'] * df['adx'] / 100
    df['momentum_score'] = (
        (df['rsi'] > 50).astype(int) + 
        (df['macd'] > df['macd_signal']).astype(int) + 
        (df['plus_di'] > df['minus_di']).astype(int)
    )
    df['price_position'] = (df['close'] - df['sma_50']) / (df['atr'] + 1e-10)
    df['trend_score'] = (
        (df['close'] > df['sma_20']).astype(int) +
        (df['sma_20'] > df['sma_50']).astype(int) +
        (df['sma_50'] > df['sma_200']).astype(int) +
        (df['adx'] > 25).astype(int)
    )
    df['rsi_zone'] = pd.cut(df['rsi'], bins=[0, 30, 45, 55, 70, 100], labels=[0, 1, 2, 3, 4]).astype(float)
    df['macd_momentum'] = df['macd_hist'] - df['macd_hist'].shift(3)
    df['close_vs_high'] = (df['high'].rolling(20).max() - df['close']) / (df['atr'] + 1e-10)
    df['close_vs_low'] = (df['close'] - df['low'].rolling(20).min()) / (df['atr'] + 1e-10)
    
    # ==================== V10 NEW FEATURES ====================
    
    # 1. Trend Strength Score (0-5)
    df['trend_strength'] = (
        (df['close'] > df['ema_5']).astype(int) +
        (df['ema_5'] > df['ema_10']).astype(int) +
        (df['ema_10'] > df['ema_20']).astype(int) +
        (df['ema_20'] > df['ema_50']).astype(int) +
        (df['adx'] > 20).astype(int)
    )
    
    # 2. Momentum Alignment
    df['momentum_alignment'] = (
        (df['rsi'] > 55).astype(int) +
        (df['macd_hist'] > 0).astype(int) +
        (df['cci'] > 50).astype(int) +
        (df['williams_r'] > -30).astype(int)
    )
    
    # 3. Volatility State
    df['volatility_sma'] = df['volatility'].rolling(50).mean()
    df['volatility_state'] = np.where(
        df['volatility'] > df['volatility_sma'] * 1.5, 2,  # High volatility
        np.where(df['volatility'] < df['volatility_sma'] * 0.5, 0, 1)  # Low, Normal
    )
    
    # 4. Price Action Patterns
    df['body'] = df['close'] - df['open']
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['body_ratio'] = abs(df['body']) / (df['high'] - df['low'] + 1e-10)
    
    # 5. Bullish/Bearish candle patterns
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['bullish_streak'] = df['is_bullish'].rolling(5).sum()
    
    # 6. Support/Resistance proximity
    df['dist_to_high20'] = (df['high'].rolling(20).max() - df['close']) / (df['atr'] + 1e-10)
    df['dist_to_low20'] = (df['close'] - df['low'].rolling(20).min()) / (df['atr'] + 1e-10)
    
    # 7. Multi-timeframe momentum (using rolling windows as proxy)
    df['rsi_5'] = df['rsi'].rolling(5).mean()
    df['rsi_20'] = df['rsi'].rolling(20).mean()
    df['rsi_trend'] = df['rsi_5'] - df['rsi_20']
    
    # 8. Breakout Detection
    df['above_bb_upper'] = (df['close'] > df['bb_upper']).astype(int)
    df['below_bb_lower'] = (df['close'] < df['bb_lower']).astype(int)
    df['bb_breakout'] = df['above_bb_upper'] - df['below_bb_lower']
    
    # 9. Price Momentum
    df['price_change_5'] = (df['close'] - df['close'].shift(5)) / (df['atr'] + 1e-10)
    df['price_change_10'] = (df['close'] - df['close'].shift(10)) / (df['atr'] + 1e-10)
    df['price_change_20'] = (df['close'] - df['close'].shift(20)) / (df['atr'] + 1e-10)
    
    # 10. Session Quality (overlap = best trading time)
    df['session_quality'] = df['is_london'].astype(int) + df['is_ny'].astype(int) + df['is_overlap'].astype(int) * 2
    
    # Cleanup temp columns
    drop_cols = ['tr0', 'tr1', 'tr2', 'tr', 'up_move', 'down_move', 'plus_dm', 'minus_dm']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
    
    return df

print("Adding V10 features...")
train_df = add_features_v10(train_df)
test_df = add_features_v10(test_df)
print(f"✓ Features added. Total columns: {len(train_df.columns)}")

Adding V10 features...
✓ Features added. Total columns: 69
✓ Features added. Total columns: 69


## 3. V10 Labeling (More Selective)

In [4]:
def create_labels_v10(df, forward_periods=60, min_pips=15, ratio=1.5):
    """
    V10 Labeling: V8-тай адилхан гэхдээ parameters-ийг тааруулах боломжтой
    
    BUY (1): Up move >= min_pips AND Up > Down * ratio
    SELL (0): Down move >= min_pips AND Down > Up * ratio
    HOLD (-1): Neither
    """
    df = df.copy()
    min_move = min_pips * 0.0001
    
    df['future_max'] = df['high'].rolling(forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    conditions = [
        (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * ratio),
        (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * ratio)
    ]
    choices = [1, 0]
    df['signal'] = np.select(conditions, choices, default=-1)
    
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# V10: Same parameters as V8 for fair comparison
train_df = create_labels_v10(train_df, forward_periods=60, min_pips=15, ratio=1.5)
test_df = create_labels_v10(test_df, forward_periods=60, min_pips=15, ratio=1.5)

# Filter BUY/SELL only
train_binary = train_df[train_df['signal'] != -1].copy()
test_binary = test_df[test_df['signal'] != -1].copy()

print(f"Train: {len(train_binary):,} (BUY/SELL only)")
print(f"Test: {len(test_binary):,}")
print(f"BUY ratio (train): {train_binary['signal'].mean()*100:.1f}%")
print(f"BUY ratio (test): {test_binary['signal'].mean()*100:.1f}%")

Train: 393,249 (BUY/SELL only)
Test: 80,302
BUY ratio (train): 49.3%
BUY ratio (test): 52.2%


## 4. Feature Selection

In [5]:
# Prepare Data
exclude_cols = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [c for c in train_binary.columns if c not in exclude_cols]

train_clean = train_binary.dropna(subset=feature_cols).copy()
test_clean = test_binary.dropna(subset=feature_cols).copy()

X_train_full = train_clean[feature_cols].values
y_train = train_clean['signal'].values
X_test_full = test_clean[feature_cols].values
y_test = test_clean['signal'].values

print(f"Features before selection: {len(feature_cols)}")

# Feature Importance based selection
selector_model = lgb.LGBMClassifier(
    n_estimators=200, max_depth=6, random_state=42, verbose=-1, device='gpu'
)
selector_model.fit(X_train_full, y_train)

# Get feature importances
importances = selector_model.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances
}).sort_values('importance', ascending=False)

# Select top features - use median threshold (like V8)
threshold = np.median(importances)
selected_features = importance_df[importance_df['importance'] >= threshold]['feature'].tolist()

print(f"Features after selection: {len(selected_features)}")
print(f"\nTop 20 Features:")
print(importance_df.head(20).to_string(index=False))

Features before selection: 63
Features after selection: 32

Top 20 Features:
       feature  importance
          hour         746
volatility_sma         685
       sma_200         548
   day_of_week         450
       ema_200         422
      bb_lower         254
      bb_upper         207
    volatility         204
           atr         201
        sma_50         192
   macd_signal         163
        ema_50         139
         ema_5         130
        rsi_20         125
price_position         116
         sma_5         105
        ema_20          98
           adx          94
        bb_std          73
          macd          70
Features after selection: 32

Top 20 Features:
       feature  importance
          hour         746
volatility_sma         685
       sma_200         548
   day_of_week         450
       ema_200         422
      bb_lower         254
      bb_upper         207
    volatility         204
           atr         201
        sma_50         192
   macd_sign

## 5. Prepare Final Data

In [6]:
# Use selected features
X_train = train_clean[selected_features].values
X_test = test_clean[selected_features].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training data: {X_train_scaled.shape}")
print(f"Test data: {X_test_scaled.shape}")

Training data: (393152, 32)
Test data: (80288, 32)


## 6. V10 Model Training (7 Diverse Models)

In [7]:
print("="*70)
print("🚀 TRAINING V10 MODELS (7 Diverse Ensemble)")
print("="*70)

models = {}

# 1. XGBoost - Primary (like V8)
models['xgb1'] = xgb.XGBClassifier(
    n_estimators=600, max_depth=6, learning_rate=0.03,
    subsample=0.8, colsample_bytree=0.8,
    reg_alpha=0.1, reg_lambda=1.0,
    min_child_weight=3,
    random_state=42, tree_method='hist', device='cuda', verbosity=0
)

# 2. XGBoost - Deeper
models['xgb2'] = xgb.XGBClassifier(
    n_estimators=400, max_depth=8, learning_rate=0.05,
    subsample=0.7, colsample_bytree=0.7,
    reg_alpha=0.05, reg_lambda=0.5,
    gamma=0.1,
    random_state=43, tree_method='hist', device='cuda', verbosity=0
)

# 3. XGBoost - Conservative
models['xgb3'] = xgb.XGBClassifier(
    n_estimators=800, max_depth=4, learning_rate=0.02,
    subsample=0.85, colsample_bytree=0.85,
    reg_alpha=0.2, reg_lambda=2.0,
    min_child_weight=5,
    random_state=44, tree_method='hist', device='cuda', verbosity=0
)

# 4. LightGBM - Primary (like V8)
models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=600, max_depth=6, learning_rate=0.03,
    subsample=0.8, colsample_bytree=0.8,
    num_leaves=31, min_child_samples=30,
    reg_alpha=0.1, reg_lambda=1.0,
    random_state=42, verbose=-1, device='gpu'
)

# 5. LightGBM - More leaves
models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.04,
    subsample=0.75, colsample_bytree=0.75,
    num_leaves=63, min_child_samples=20,
    reg_alpha=0.05, reg_lambda=0.5,
    random_state=45, verbose=-1, device='gpu'
)

# 6. CatBoost - Primary (like V8)
models['cat1'] = CatBoostClassifier(
    iterations=600, depth=6, learning_rate=0.03,
    l2_leaf_reg=3.0, random_strength=0.5,
    bagging_temperature=0.5,
    random_seed=42, task_type='GPU', devices='0', verbose=False
)

# 7. CatBoost - Deeper
models['cat2'] = CatBoostClassifier(
    iterations=500, depth=8, learning_rate=0.04,
    l2_leaf_reg=2.0, random_strength=0.3,
    bagging_temperature=0.3,
    random_seed=46, task_type='GPU', devices='0', verbose=False
)

# Train all models
predictions = {}
probabilities = {}

for name, model in models.items():
    print(f"  Training {name.upper()}...", end=" ")
    model.fit(X_train_scaled, y_train)
    predictions[name] = model.predict(X_test_scaled)
    probabilities[name] = model.predict_proba(X_test_scaled)
    test_acc = accuracy_score(y_test, predictions[name])
    print(f"Test: {test_acc*100:.2f}%")

print(f"\n✓ All {len(models)} models trained")

🚀 TRAINING V10 MODELS (7 Diverse Ensemble)
  Training XGB1... Test: 50.73%
  Training XGB2... Test: 50.73%
  Training XGB2... Test: 50.11%
  Training XGB3... Test: 50.11%
  Training XGB3... Test: 50.82%
  Training LGB1... Test: 50.82%
  Training LGB1... Test: 50.61%
  Training LGB2... Test: 50.61%
  Training LGB2... Test: 49.88%
  Training CAT1... Test: 49.88%
  Training CAT1... Test: 51.28%
  Training CAT2... Test: 51.28%
  Training CAT2... Test: 50.35%

✓ All 7 models trained
Test: 50.35%

✓ All 7 models trained


## 7. V10 Ensemble (Weighted + Agreement)

In [8]:
print("="*70)
print("🔧 BUILDING V10 ENSEMBLE")
print("="*70)

# Accuracy-based weights
accuracies = {name: accuracy_score(y_test, predictions[name]) for name in models.keys()}
total_acc = sum(accuracies.values())
weights = {name: acc / total_acc for name, acc in accuracies.items()}

print("Model Weights (accuracy-based):")
for name, w in sorted(weights.items(), key=lambda x: -x[1]):
    print(f"  {name}: {w:.4f} (acc: {accuracies[name]*100:.2f}%)")

# Weighted Ensemble Probability
final_proba = np.zeros_like(probabilities['xgb1'])
for name, w in weights.items():
    final_proba += w * probabilities[name]

buy_prob = final_proba[:, 1] * 100

# Model Agreement Analysis
all_preds = np.array([predictions[name] for name in models.keys()])
buy_votes = np.sum(all_preds == 1, axis=0)
sell_votes = np.sum(all_preds == 0, axis=0)

# Agreement levels
all_agree_buy = buy_votes == len(models)  # 7/7
strong_buy = buy_votes >= 6  # 6+/7
majority_buy = buy_votes >= 5  # 5+/7

all_agree_sell = sell_votes == len(models)  # 7/7
strong_sell = sell_votes >= 6  # 6+/7

print(f"\nAgreement Analysis:")
print(f"  All 7 agree BUY: {all_agree_buy.sum():,}")
print(f"  6+ agree BUY: {strong_buy.sum():,}")
print(f"  5+ agree BUY: {majority_buy.sum():,}")
print(f"  All 7 agree SELL: {all_agree_sell.sum():,}")
print(f"  6+ agree SELL: {strong_sell.sum():,}")

🔧 BUILDING V10 ENSEMBLE
Model Weights (accuracy-based):
  cat1: 0.1449 (acc: 51.28%)
  xgb3: 0.1437 (acc: 50.82%)
  xgb1: 0.1434 (acc: 50.73%)
  lgb1: 0.1431 (acc: 50.61%)
  cat2: 0.1423 (acc: 50.35%)
  xgb2: 0.1416 (acc: 50.11%)
  lgb2: 0.1410 (acc: 49.88%)

Agreement Analysis:
  All 7 agree BUY: 19,271
  6+ agree BUY: 26,961
  5+ agree BUY: 33,905
  All 7 agree SELL: 20,391
  6+ agree SELL: 28,233


## 8. V10 Confidence Calculation

In [9]:
print("="*70)
print("🎯 CALCULATING V10 CONFIDENCE")
print("="*70)

# Base confidence from ensemble probability
confidence = buy_prob.copy()

# V10 Agreement Bonus (7 models)
# All 7 agree = +7 bonus
confidence[all_agree_buy] = np.minimum(confidence[all_agree_buy] + 7, 100)
# 6+ agree = +4 bonus
confidence[strong_buy & ~all_agree_buy] = np.minimum(confidence[strong_buy & ~all_agree_buy] + 4, 100)
# 5+ agree = +2 bonus
confidence[majority_buy & ~strong_buy] = np.minimum(confidence[majority_buy & ~strong_buy] + 2, 100)

print(f"Confidence distribution:")
print(f"  >= 90%: {(confidence >= 90).sum():,}")
print(f"  >= 85%: {(confidence >= 85).sum():,}")
print(f"  >= 80%: {(confidence >= 80).sum():,}")
print(f"  >= 75%: {(confidence >= 75).sum():,}")
print(f"  >= 70%: {(confidence >= 70).sum():,}")

🎯 CALCULATING V10 CONFIDENCE
Confidence distribution:
  >= 90%: 17
  >= 85%: 64
  >= 80%: 255
  >= 75%: 826
  >= 70%: 2,606


## 9. V10 Results

In [10]:
print("="*70)
print("📊 V10 ENSEMBLE RESULTS")
print("="*70)

print(f"\n{'Confidence':>12} | {'BUY Signals':>12} | {'Correct':>10} | {'Accuracy':>10}")
print("-"*60)

v10_results = {}
for conf in [50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
    mask = confidence >= conf
    if mask.sum() > 0:
        signals = mask.sum()
        correct = y_test[mask].sum()  # BUY correct
        acc = correct / signals * 100
        v10_results[conf] = {'signals': signals, 'correct': int(correct), 'accuracy': acc}
        marker = " 🎯" if acc >= 67.6 else ""  # V8 75% benchmark
        print(f"{conf:>10}%+ | {signals:>12} | {correct:>10.0f} | {acc:>9.1f}%{marker}")

print("\n" + "-"*60)
print("🎯 = V8 75%+ benchmark (67.6%) тай тэнцүү эсвэл илүү")

📊 V10 ENSEMBLE RESULTS

  Confidence |  BUY Signals |    Correct |   Accuracy
------------------------------------------------------------
        50%+ |        40109 |      21040 |      52.5%
        55%+ |        28805 |      15266 |      53.0%
        60%+ |        19866 |      10654 |      53.6%
        65%+ |         9082 |       4855 |      53.5%
        70%+ |         2606 |       1485 |      57.0%
        75%+ |          826 |        501 |      60.7%
        80%+ |          255 |        183 |      71.8% 🎯
        85%+ |           64 |         62 |      96.9% 🎯
        90%+ |           17 |         17 |     100.0% 🎯

------------------------------------------------------------
🎯 = V8 75%+ benchmark (67.6%) тай тэнцүү эсвэл илүү


## 10. V10 vs V8 Comparison

In [1]:
# Load V8 for comparison
v8_dir = BASE_DIR / 'models' / 'signal_generator_v8'

try:
    v8_models = {}
    for name in ['xgb1', 'xgb2', 'lgb1', 'lgb2', 'cat']:
        v8_models[name] = joblib.load(v8_dir / f'{name}_v8.joblib')
    
    v8_scaler = joblib.load(v8_dir / 'scaler_v8.joblib')
    v8_feature_cols = joblib.load(v8_dir / 'feature_cols_v8.joblib')
    v8_weights = joblib.load(v8_dir / 'weights_v8.joblib')
    
    # Prepare V8 test data
    missing = [c for c in v8_feature_cols if c not in test_clean.columns]
    for c in missing:
        test_clean[c] = 0
    
    X_test_v8 = test_clean[v8_feature_cols].values
    X_test_v8_scaled = v8_scaler.transform(X_test_v8)
    
    # V8 predictions
    v8_proba = {}
    v8_preds = {}
    for name, model in v8_models.items():
        v8_preds[name] = model.predict(X_test_v8_scaled)
        v8_proba[name] = model.predict_proba(X_test_v8_scaled)
    
    # V8 ensemble
    v8_final_proba = np.zeros_like(v8_proba['xgb1'])
    for name, w in v8_weights.items():
        v8_final_proba += w * v8_proba[name]
    
    v8_buy_prob = v8_final_proba[:, 1] * 100
    v8_all_agree = np.all([v8_preds[name] == 1 for name in v8_models.keys()], axis=0)
    v8_most_agree = np.sum([v8_preds[name] == 1 for name in v8_models.keys()], axis=0) >= 4
    v8_confidence = v8_buy_prob.copy()
    v8_confidence[v8_all_agree] = np.minimum(v8_confidence[v8_all_agree] + 5, 100)
    v8_confidence[v8_most_agree & ~v8_all_agree] = np.minimum(v8_confidence[v8_most_agree & ~v8_all_agree] + 2, 100)
    
    print("="*80)
    print("📊 V10 vs V8 COMPARISON")
    print("="*80)
    print(f"\n{'Threshold':>10} | {'V8 Sig':>8} | {'V8 Acc':>8} | {'V10 Sig':>8} | {'V10 Acc':>8} | {'Winner':>10}")
    print("-"*75)
    
    for conf in [70, 75, 80, 85, 90]:
        # V8
        v8_mask = v8_confidence >= conf
        v8_sig = v8_mask.sum()
        v8_acc = y_test[v8_mask].mean() * 100 if v8_sig > 0 else 0
        
        # V10
        if conf in v10_results:
            v10_sig = v10_results[conf]['signals']
            v10_acc = v10_results[conf]['accuracy']
        else:
            v10_sig = 0
            v10_acc = 0
        
        if v10_acc > v8_acc + 1:
            winner = "V10 ✓"
        elif v8_acc > v10_acc + 1:
            winner = "V8 ✓"
        else:
            winner = "≈ TIE"
        
        print(f"{conf:>8}%+ | {v8_sig:>8} | {v8_acc:>7.1f}% | {v10_sig:>8} | {v10_acc:>7.1f}% | {winner:>10}")
    
    print("\n" + "="*80)
    print("📋 V8 ORIGINAL BENCHMARK (from V8 notebook):")
    print("   75%+: 627 signals, 67.6% accuracy")
    print("   85%+: 48 signals, 93.8% accuracy")
    print("   90%+: 8 signals, 100% accuracy")
    print("="*80)

except Exception as e:
    print(f"V8 models not found: {e}")
    print("\n📋 V8 BENCHMARK:")
    print("   75%+: 627 signals, 67.6% accuracy")
    print("   85%+: 48 signals, 93.8% accuracy")

NameError: name 'BASE_DIR' is not defined

## 11. Statistical Significance

In [2]:
from scipy import stats

print("="*70)
print("📊 STATISTICAL SIGNIFICANCE (95% Confidence Interval)")
print("="*70)

def confidence_interval(n_success, n_total, confidence=0.95):
    """Wilson score interval for proportion"""
    if n_total == 0:
        return 0, 0, 0
    p = n_success / n_total
    z = stats.norm.ppf((1 + confidence) / 2)
    
    denominator = 1 + z**2 / n_total
    center = (p + z**2 / (2 * n_total)) / denominator
    spread = z * np.sqrt((p * (1 - p) + z**2 / (4 * n_total)) / n_total) / denominator
    
    lower = max(0, center - spread)
    upper = min(1, center + spread)
    return p, lower, upper

print(f"\n{'Threshold':>10} | {'Signals':>8} | {'Accuracy':>10} | {'95% CI':>20} | {'Reliable?':>12}")
print("-"*75)

for conf in [70, 75, 80, 85, 90]:
    if conf in v10_results:
        r = v10_results[conf]
        n_total = r['signals']
        n_success = r['correct']
        
        acc, lower, upper = confidence_interval(n_success, n_total)
        ci_str = f"[{lower*100:.1f}% - {upper*100:.1f}%]"
        
        if n_total >= 100 and lower > 0.60:
            reliable = "✅ RELIABLE"
        elif n_total >= 50 and lower > 0.55:
            reliable = "⚡ MODERATE"
        elif n_total < 30:
            reliable = "⚠️ TOO FEW"
        else:
            reliable = "❌ WEAK"
        
        print(f"{conf:>8}%+ | {n_total:>8} | {acc*100:>9.1f}% | {ci_str:>20} | {reliable:>12}")

📊 STATISTICAL SIGNIFICANCE (95% Confidence Interval)

 Threshold |  Signals |   Accuracy |               95% CI |    Reliable?
---------------------------------------------------------------------------


NameError: name 'v10_results' is not defined

## 12. Overfit Analysis

In [3]:
print("="*70)
print("🔍 OVERFIT ANALYSIS: Train vs Test")
print("="*70)

# Train predictions
train_proba_dict = {}
train_preds_dict = {}
for name, model in models.items():
    train_proba_dict[name] = model.predict_proba(X_train_scaled)
    train_preds_dict[name] = model.predict(X_train_scaled)

# Train ensemble
train_final_proba = np.zeros_like(train_proba_dict['xgb1'])
for name, w in weights.items():
    train_final_proba += w * train_proba_dict[name]

train_buy_prob = train_final_proba[:, 1] * 100

# Agreement bonus
train_all_preds = np.array([train_preds_dict[name] for name in models.keys()])
train_buy_votes = np.sum(train_all_preds == 1, axis=0)
train_all_agree = train_buy_votes == len(models)
train_strong_buy = train_buy_votes >= 6
train_majority_buy = train_buy_votes >= 5

train_confidence = train_buy_prob.copy()
train_confidence[train_all_agree] = np.minimum(train_confidence[train_all_agree] + 7, 100)
train_confidence[train_strong_buy & ~train_all_agree] = np.minimum(train_confidence[train_strong_buy & ~train_all_agree] + 4, 100)
train_confidence[train_majority_buy & ~train_strong_buy] = np.minimum(train_confidence[train_majority_buy & ~train_strong_buy] + 2, 100)

print(f"\n{'Threshold':>10} | {'Train Sig':>10} | {'Train Acc':>10} | {'Test Sig':>10} | {'Test Acc':>10} | {'Diff':>10}")
print("-"*80)

for conf in [70, 75, 80, 85, 90]:
    # Train
    train_mask = train_confidence >= conf
    train_sig = train_mask.sum()
    train_acc = y_train[train_mask].mean() * 100 if train_sig > 0 else 0
    
    # Test
    test_mask = confidence >= conf
    test_sig = test_mask.sum()
    test_acc = y_test[test_mask].mean() * 100 if test_sig > 0 else 0
    
    diff = train_acc - test_acc
    
    print(f"{conf:>8}%+ | {train_sig:>10} | {train_acc:>9.1f}% | {test_sig:>10} | {test_acc:>9.1f}% | {diff:>+9.1f}%")

🔍 OVERFIT ANALYSIS: Train vs Test


NameError: name 'models' is not defined

## 13. Final Summary

In [4]:
print("="*80)
print("📋 V10 FINAL SUMMARY")
print("="*80)

# Find best config
reliable_results = {k: v for k, v in v10_results.items() if v['signals'] >= 50}
if reliable_results:
    best_reliable = max(reliable_results.items(), key=lambda x: x[1]['accuracy'])
    print(f"\n🏆 BEST RELIABLE: {best_reliable[0]}% threshold")
    print(f"   Accuracy: {best_reliable[1]['accuracy']:.1f}%")
    print(f"   Signals: {best_reliable[1]['signals']}")

print("\n💡 V10 CONFIGURATIONS:")
for conf in [70, 75, 80, 85, 90]:
    if conf in v10_results and v10_results[conf]['signals'] > 0:
        r = v10_results[conf]
        beat_v8 = "✓" if r['accuracy'] >= 67.6 else ""
        print(f"   {conf}%+: {r['accuracy']:.1f}% acc, {r['signals']} signals {beat_v8}")

print("\n📊 V8 BENCHMARK:")
print("   75%+: 67.6% acc, 627 signals")
print("   85%+: 93.8% acc, 48 signals")

print("\n🎯 V10 IMPROVEMENTS:")
print("   - 7 diverse models (vs V8's 5)")
print("   - Enhanced agreement bonus")
print("   - Additional pattern features")

📋 V10 FINAL SUMMARY


NameError: name 'v10_results' is not defined

## 14. Save V10 Models

In [5]:
print("Saving V10 Models...")

# Save base models
for name, model in models.items():
    joblib.dump(model, MODEL_DIR / f'{name}_v10.joblib')

# Save preprocessing
joblib.dump(scaler, MODEL_DIR / 'scaler_v10.joblib')
joblib.dump(selected_features, MODEL_DIR / 'feature_cols_v10.joblib')
joblib.dump(weights, MODEL_DIR / 'weights_v10.joblib')

# Save config
config = {
    'version': 'v10',
    'mode': 'BUY_vs_SELL_7_Ensemble',
    'features': len(selected_features),
    'models': list(models.keys()),
    'weights': weights,
    'accuracies': accuracies,
    'best_threshold': best_reliable[0] if reliable_results else 75
}
joblib.dump(config, MODEL_DIR / 'config_v10.joblib')

print(f"✅ V10 Models Saved to {MODEL_DIR}")
print(f"   - {len(models)} Base Models")
print(f"   - Scaler & Feature List")
print(f"   - Weights & Config")

Saving V10 Models...


NameError: name 'models' is not defined